<a href="https://colab.research.google.com/github/jeguns/EP7173/blob/main/Unidad%2002/02_Ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El Organismo Supervisor de la Inversión en Energía y Minería es una institución pública adscrita a la Presidencia del Consejo de Ministros del Perú y está encargada de supervisar que las empresas eléctricas, las de combustibles y las del régimen general de Minería del Perú cumplan las normas legales de las actividades que desarrollan.

En [Plataforma Nacional de Datos Abiertos](https://www.datosabiertos.gob.pe/dataset/registro-de-hidrocarburos-l%C3%ADquidos-setiembre-2023-osinergmin/resource/33def057-dd58-4bbe#{view-graph:{graphOptions:{hooks:{processOffset:{},bindEvents:{}}}},graphOptions:{hooks:{processOffset:{},bindEvents:{}}}}
) se tiene disponible el registro de grifos y estaciones de servicios, actualizado al 12 de setiembre del 2023.


1. Leer el archivo de datos y e indicar cuál es el tipo de dato que contiene cada atributo.

In [ ]:
import pandas as pd

In [ ]:
datos = pd.read_excel('/content/Grifos y Estaciones de Servicios.xlsx')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   No                       3359 non-null   int64  
 1   EXPEDIENTE               3359 non-null   int64  
 2   CODIGO OSINERGMIN        3359 non-null   int64  
 3   REGISTRO                 3359 non-null   object 
 4   RUC                      3359 non-null   int64  
 5   RAZON SOCIAL             3359 non-null   object 
 6   DIRECCION OPERATIVA      3359 non-null   object 
 7   DEPARTAMENTO             3359 non-null   object 
 8   PROVINCIA                3359 non-null   object 
 9   DISTRITO                 3359 non-null   object 
 10  TIPO DE ESTABLECIMIENTO  3359 non-null   object 
 11  TANQUE 1                 3359 non-null   object 
 12  TANQUE 2                 3239 non-null   object 
 13  TANQUE 3                 2463 non-null   object 
 14  TANQUE 4                

2. Seleccione los establecimientos que son de tipo "ESTACIONES DE SERVICIOS" cuya fecha de emisión de registro se encuentre entre el 10 de febrero del 2022 y el 15 de mayo del 2023. Una vez realizado el filtro, seleccionar las columnas RUC, RAZON SOCIAL, TIPO DE ESTABLECIMIENTO, FEC. DE EMISION Y REPRESENTANTE, ordenadas de la fecha más reciente a la antigua, y exportar el data frame resultante en datos1.csv

In [ ]:
datos[['TIPO DE ESTABLECIMIENTO']].value_counts()

TIPO DE ESTABLECIMIENTO                
ESTACIONES DE SERVICIOS                    1782
PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS    1561
GRIFO VIA PUBLICA                            16
dtype: int64

In [ ]:
datos['FECHA'] = pd.to_datetime(datos['FEC. EMISION'], format = '%d_%m_%y')
datos[['FECHA','FEC. EMISION']].head()

,FECHA,FEC. EMISION
0,2023-03-29,29_03_23
1,2023-06-19,19_06_23
2,2022-08-24,24_08_22
3,2023-06-05,5_06_23
4,2023-08-09,9_08_23


In [ ]:
fecha_inicio = pd.to_datetime('2021-02-10')
fecha_fin    = pd.to_datetime('2023-05-15')
datos1       = datos.loc[(datos['TIPO DE ESTABLECIMIENTO'] == "ESTACIONES DE SERVICIOS") & (datos['FECHA'] >= fecha_inicio) & (datos['FECHA'] <= fecha_fin)]
datos1       = datos1[['RUC','RAZON SOCIAL','TIPO DE ESTABLECIMIENTO','FECHA','REPRESENTANTE']].sort_values(by='FECHA', ascending=False).reset_index()
datos1.to_csv('datos1.csv', index=False)

3. Seleccionar los tipos de establecimiento que son PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS que se hayan registrado a inicio o a fin de mes del año 2023. Luego del filtro, seleccionar los atributos RUC, RAZON SOCIAL, DEPARTAMENTO, PROVINCIA, DISTRITO y una columna indicando si el registro se hizo a inicio o fin de mes, y exportar el data frame resultante como datos2.xlsx

In [ ]:
import numpy as np
datos['ANHO']       = datos['FECHA'].dt.year
datos['INICIA_MES'] = datos['FECHA'].dt.is_month_start
datos['FIN_MES']    = datos['FECHA'].dt.is_month_end
datos['MOMENTO']    = np.where(datos['INICIA_MES'] == True, 'INICIO DE MES','FIN DE MES')
datos2 = datos.loc[
    (datos['TIPO DE ESTABLECIMIENTO'] == "PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS") &
     (datos['ANHO'] == 2023) &
      ((datos['INICIA_MES'] == True) | (datos['FIN_MES'] == True))]
datos2 = datos2[['RUC','RAZON SOCIAL','DEPARTAMENTO','PROVINCIA','DISTRITO','MOMENTO']]
datos2.to_excel('datos2.xlsx', index=False)

In [ ]:
datos['INICIA_TRIM']  = datos['FECHA'].dt.is_quarter_start
datos['FIN_TRIM']  = datos['FECHA'].dt.is_quarter_end
datos['INICIA_ANHO']  = datos['FECHA'].dt.is_year_start
datos['FIN_ANHO']  = datos['FECHA'].dt.is_year_end

4. Obtener la capacidad total promedio de combustibles líquidos de los registros realizados los días lunes y que se encuentren dentro de los 5 primeros días del mes.

In [ ]:
datos['DIA_NOMBRE'] = datos['FECHA'].dt.day_name()
datos['DIA_SEMANA'] = datos['FECHA'].dt.weekday + 1
datos['DIA_MES']    = datos['FECHA'].dt.day
datos3              = datos.loc[(datos['DIA_NOMBRE']=="Monday") & (datos['DIA_MES']<=5)]
datos3['CAP.TOTAL CL (gln)'].mean()

16351.95652173913

In [ ]:
datos.loc[(datos['FECHA'].dt.day_name() == "Monday") & (datos['FECHA'].dt.day <= 5)]['CAP.TOTAL CL (gln)'].mean()

16351.95652173913

In [ ]:
datos.loc[(datos['FECHA'].dt.weekday == 0) & (datos['FECHA'].dt.day <= 5)]['CAP.TOTAL CL (gln)'].mean()

16351.95652173913

5. Seleccionar el RUC, Codigo OSINERGMIN, Provincia y Fecha de registro de las instancias correspondientes a la provincia de Lima o Arequipa, y hayan sido hechos en el último dia de los meses que tienen 31 días.

In [ ]:
datos['DIAS_MES']   = datos['FECHA'].dt.days_in_month ####
datos.loc[(datos['PROVINCIA'].isin(['LIMA','AREQUIPA'])) & (datos['DIAS_MES'] == 31) & (datos['FIN_MES'] == True)][['RUC','CODIGO OSINERGMIN','PROVINCIA','FECHA']]

,RUC,CODIGO OSINERGMIN,PROVINCIA,FECHA
268,10292260003,92944,AREQUIPA,2022-07-31
412,20311851650,38135,AREQUIPA,2016-05-31
1921,20419028330,17856,LIMA,2017-10-31
2443,20503840121,19984,LIMA,2016-07-31
2792,20388308134,96646,LIMA,2020-08-31
3215,20606440597,121053,LIMA,2022-05-31


In [ ]:
datos.loc[((datos['PROVINCIA'] == "LIMA") | (datos['PROVINCIA'] == "AREQUIPA")) & (datos['DIAS_MES'] == 31) & (datos['FIN_MES'] == True)][['RUC','CODIGO OSINERGMIN','PROVINCIA','FECHA']]

,RUC,CODIGO OSINERGMIN,PROVINCIA,FECHA
268,10292260003,92944,AREQUIPA,2022-07-31
412,20311851650,38135,AREQUIPA,2016-05-31
1921,20419028330,17856,LIMA,2017-10-31
2443,20503840121,19984,LIMA,2016-07-31
2792,20388308134,96646,LIMA,2020-08-31
3215,20606440597,121053,LIMA,2022-05-31


6. Seleccionar el tipo de establecimoento y la fecha de los registros que hayan sido realizados en mayo, noviembre, diciembre o enero del año 2021 o 2023.

In [ ]:
datos['MES']  = datos['FECHA'].dt.month
datos['NMES'] = datos['FECHA'].dt.month_name()
datos.loc[(datos['MES'].isin([5,11,12])) & datos['ANHO'].isin([2021,2023])][['TIPO DE ESTABLECIMIENTO','FECHA']]

,TIPO DE ESTABLECIMIENTO,FECHA
45,ESTACIONES DE SERVICIOS,2023-05-31
46,ESTACIONES DE SERVICIOS,2023-05-13
51,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-04
59,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2021-05-25
96,ESTACIONES DE SERVICIOS,2023-05-16
...,...,...
3277,ESTACIONES DE SERVICIOS,2023-05-27
3295,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-10
3332,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-10
3350,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2021-05-12


In [ ]:
datos.loc[(datos['FECHA'].dt.month.isin([5,11,12])) & datos['FECHA'].dt.year.isin([2021,2023])][['TIPO DE ESTABLECIMIENTO','FECHA']]

,TIPO DE ESTABLECIMIENTO,FECHA
45,ESTACIONES DE SERVICIOS,2023-05-31
46,ESTACIONES DE SERVICIOS,2023-05-13
51,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-04
59,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2021-05-25
96,ESTACIONES DE SERVICIOS,2023-05-16
...,...,...
3277,ESTACIONES DE SERVICIOS,2023-05-27
3295,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-10
3332,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-10
3350,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2021-05-12


In [ ]:
datos.loc[(datos['FECHA'].dt.month_name().isin(['May','November','December'])) & datos['FECHA'].dt.year.isin([2021,2023])][['TIPO DE ESTABLECIMIENTO','FECHA']]

,TIPO DE ESTABLECIMIENTO,FECHA
45,ESTACIONES DE SERVICIOS,2023-05-31
46,ESTACIONES DE SERVICIOS,2023-05-13
51,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-04
59,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2021-05-25
96,ESTACIONES DE SERVICIOS,2023-05-16
...,...,...
3277,ESTACIONES DE SERVICIOS,2023-05-27
3295,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-10
3332,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2023-05-10
3350,PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS,2021-05-12


7. Seleccionar el departamento, provincia, distrito y fecha de los registros correspondientes al segundo semestre del año 2022 y que no sean del departamento de Lima. Luego, obtener e interpretar la moda de departamento, provincia y distrito.

In [ ]:
import math
datos['TRIMESTRE'] =  datos['FECHA'].dt.quarter
datos['SEMESTRE']  =  datos['TRIMESTRE'].apply(lambda x: math.ceil(x/2))
datos8 = datos.loc[(datos['SEMESTRE'] == 2) & (datos['ANHO'] == 2022) & (datos['DEPARTAMENTO'] != "LIMA")][['DEPARTAMENTO','PROVINCIA','DISTRITO','FECHA']]
datos8

,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA
2,LA LIBERTAD,TRUJILLO,MOCHE,2022-08-24
8,MADRE DE DIOS,TAMBOPATA,INAMBARI,2022-09-22
25,PIURA,AYABACA,AYABACA,2022-11-10
31,AREQUIPA,AREQUIPA,AREQUIPA,2022-10-05
35,AYACUCHO,HUAMANGA,JESUS NAZARENO,2022-10-05
...,...,...,...,...
3319,CUSCO,CUSCO,SANTIAGO,2022-10-03
3329,UCAYALI,CORONEL PORTILLO,CAMPOVERDE,2022-08-15
3331,AREQUIPA,AREQUIPA,SOCABAYA,2022-09-07
3352,AMAZONAS,RODRIGUEZ DE MENDOZA,OMIA,2022-11-17


In [ ]:
datos7 = datos.loc[(datos['FECHA'].dt.quarter.isin([3,4])) & (datos['ANHO'] == 2022) & (datos['DEPARTAMENTO'] != "LIMA")][['DEPARTAMENTO','PROVINCIA','DISTRITO','FECHA']]
datos7

,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA
2,LA LIBERTAD,TRUJILLO,MOCHE,2022-08-24
8,MADRE DE DIOS,TAMBOPATA,INAMBARI,2022-09-22
25,PIURA,AYABACA,AYABACA,2022-11-10
31,AREQUIPA,AREQUIPA,AREQUIPA,2022-10-05
35,AYACUCHO,HUAMANGA,JESUS NAZARENO,2022-10-05
...,...,...,...,...
3319,CUSCO,CUSCO,SANTIAGO,2022-10-03
3329,UCAYALI,CORONEL PORTILLO,CAMPOVERDE,2022-08-15
3331,AREQUIPA,AREQUIPA,SOCABAYA,2022-09-07
3352,AMAZONAS,RODRIGUEZ DE MENDOZA,OMIA,2022-11-17


In [ ]:
datos7.DEPARTAMENTO.value_counts()

PUNO                       33
CUSCO                      26
AYACUCHO                   25
AREQUIPA                   23
SAN MARTIN                 19
LAMBAYEQUE                 16
UCAYALI                    13
PIURA                      12
CAJAMARCA                  12
LORETO                     12
LA LIBERTAD                10
JUNIN                       8
AMAZONAS                    7
MADRE DE DIOS               6
APURIMAC                    6
HUANUCO                     6
ANCASH                      5
HUANCAVELICA                4
TACNA                       4
ICA                         3
MOQUEGUA                    2
PROV. CONST. DEL CALLAO     2
PASCO                       2
Name: DEPARTAMENTO, dtype: int64

In [ ]:
datos7.PROVINCIA.value_counts()

AREQUIPA            12
CORONEL PORTILLO    12
MAYNAS              10
LA CONVENCION       10
HUAMANGA            10
                    ..
CASTILLA             1
PARURO               1
CHEPEN               1
OXAPAMPA             1
MANU                 1
Name: PROVINCIA, Length: 102, dtype: int64

In [ ]:
datos7.PROVINCIA.mode()

0            AREQUIPA
1    CORONEL PORTILLO
Name: PROVINCIA, dtype: object

In [ ]:
import statistics
statistics.mode(datos7["PROVINCIA"].tolist()) # Solo devuelve el primero

'AREQUIPA'

In [ ]:
datos7.DISTRITO.value_counts()

JULIACA               10
AYACUCHO               5
MANANTAY               5
ANCO                   4
PUNO                   4
                      ..
QUINUA                 1
MARISCAL BENAVIDES     1
HUACRACHUCO            1
CHONGOYAPE             1
MADRE DE DIOS          1
Name: DISTRITO, Length: 174, dtype: int64

8. Seleccionar los registros de los 50 primeros días del año 2023 y cuyo término de vigencia no sea indefinido.

In [ ]:
datos['ANHO']     = datos['FECHA'].dt.year
datos['DIA_ANHO'] = datos['FECHA'].dt.day_of_year
datos.loc[(datos['FECHA'].dt.year == 2023) & (datos['FECHA'].dt.day_of_year <= 50) & (datos['TÃ‰RMINO DE VIGENCIA'] != "INDEFINIDO")]['FECHA']

122    2023-01-18
964    2023-01-06
1476   2023-01-11
1482   2023-01-27
1615   2023-01-26
1829   2023-02-14
2158   2023-02-18
2639   2023-01-21
2662   2023-01-10
2755   2023-01-05
3234   2023-02-02
Name: FECHA, dtype: datetime64[ns]

9. Seleccionar los registros que hayan sido realizados al menos 50 días antes de la fecha actual. Luego, obtener el promedio de días de anticipación.

In [ ]:
from datetime import datetime

datos['ANTICIPACION'] = (datetime.now() - datos['FECHA']).dt.days
datos[['ANTICIPACION','FECHA']].sort_values(by='FECHA', ascending=False)

,ANTICIPACION,FECHA
2520,1,2023-09-12
661,2,2023-09-11
866,4,2023-09-09
3130,5,2023-09-08
1694,5,2023-09-08
...,...,...
977,8500,2000-06-05
471,8639,2000-01-18
493,8771,1999-09-08
1343,8986,1999-02-05


In [ ]:
datos.ANTICIPACION.mean()

1119.4548972908603

10. Seleccionar los registros que hayan sido realizados a partir del 28 de julio del 2023, indicando el RUC, DEPARTAMENTO, FECHA y días desde la fecha de registro.

In [ ]:
datos['DIAS_DFP'] = (datos['FECHA'] - datetime(2023, 7, 28)).dt.days
datos.loc[(datos['DIAS_DFP'] > 0)][['RUC','DEPARTAMENTO','FECHA','DIAS_DFP']]

,RUC,DEPARTAMENTO,FECHA,DIAS_DFP
4,10039009221,SAN MARTIN,2023-08-09,12
10,20600624548,CUSCO,2023-08-27,30
27,20379038761,LIMA,2023-08-15,18
37,20603597126,PIURA,2023-09-01,35
42,20600361466,CUSCO,2023-08-05,8
...,...,...,...,...
3290,10018558616,PUNO,2023-08-31,34
3303,20486274396,JUNIN,2023-08-10,13
3355,20600454731,JUNIN,2023-08-10,13
3356,20600626109,HUANCAVELICA,2023-08-01,4


In [ ]:
import datetime

dt = datetime.date(2023, 9, 16)
day_name = dt.day_name()

AttributeError: ignored

In [ ]:
import datetime

dt = datetime.date(2023, 9, 16)
day_of_week = dt.weekday()